In [1]:
from gensim.models import Word2Vec
from eunjeon import Mecab
from korean_romanizer import *
import jellyfish
from pykospacing import spacing

tagger = Mecab()

model = Word2Vec.load('psychology.model')

In [2]:
text_list=[]
with open('analysis_token.txt', 'r',encoding='utf-8') as f:
    for line in f:
        text_list.append(line.rstrip())

In [3]:
except_words=[]
with open('except_word.txt', 'r',encoding='utf-8') as f:
    except_words = []
    for line in f:
        except_words.append(line.rstrip())

In [6]:
# MAIN




def find_error_word(): #오류가 있는지 확인
    error_word=[] #오류단어를 저장
    for i in jamak_nouns:
        if i not in text_list:
            error_word.append(i)
    return error_word

def nearby_error_word(): #오류 단어 앞뒤의 3단어 뽑기
    global check_nouns
    check_nouns=[]#오류단어 앞뒤의 단어 저장
    if error_word==[]:
        return []
    else:
        for i in range(len(error_word)):
            for j in range(len(jamak_nouns)):
                if error_word[i] == jamak_nouns[j]:
                    check_nouns_list=[]
                    if j == len(jamak_nouns)-1:
                        try:
                            check_nouns_list.append(jamak_nouns[j-1])
                            check_nouns_list.append(jamak_nouns[j-2])
                            check_nouns_list.append(jamak_nouns[j-3])
                            check_nouns_list.append(jamak_nouns[j-4])
                        except:
                            pass
                    elif j == len(jamak_nouns)-2:
                        try:
                            check_nouns_list.append(jamak_nouns[j-1])
                            check_nouns_list.append(jamak_nouns[j+1])
                            check_nouns_list.append(jamak_nouns[j-2])
                            check_nouns_list.append(jamak_nouns[j-3])
                        except:
                            pass
                    else:
                        try:
                            check_nouns_list.append(jamak_nouns[j-1])
                            check_nouns_list.append(jamak_nouns[j+1])
                            check_nouns_list.append(jamak_nouns[j-2])
                            check_nouns_list.append(jamak_nouns[j+2])
                        except:
                            pass
                    check_nouns.append(check_nouns_list)    
    return check_nouns

def check_word_list(): #오류 단어 근처의 단어에 대해 word2vec으로 학습한 연관성 높은 단어의 리스트를 출력
    word_list=[]
    global model_result
    if check_nouns==[]:
        return []

    else:
        for i in range(len(check_nouns)):
            list_result=[]
            for j in check_nouns[i]:
                try:
                    model_result=model.wv.most_similar(j,topn=100)
                    for k in model_result:
                        if k[0] not in except_words:
                            list_result.append(k[0])
                except:
                    pass
            word_list.append(list_result)
    
    return word_list


def romanizing(): #word_list의 한글 발음을 로마자로 변환
    global pronounce
    pronounce=[] # word_list의 발음을 저장.
    if word_list == []:
        return []
    else:
        for i in range(len(word_list)):
            pronounce_list=[]
            for j in range(len(word_list[i])):
                try:
                    a=Romanizer(word_list[i][j])
                    pronounce_list.append(a.romanize())
                except:
                    pass
            pronounce.append(pronounce_list)
    return pronounce


def similarity(): #error word와 word list 단어의 발음 유사도 측정
    probability=[]
    if pronounce == []:
        return []
    else:
        for e in range(len(error_word)):
            a=Romanizer(error_word[e]).romanize()
            prob=[]
            prob1=[]
            prob2=[]
            prob3=[]
            for j in range(len(pronounce[e])):
                prob1.append(jellyfish.jaro_winkler_similarity(a, pronounce[e][j]))
                prob2.append(jellyfish.jaro_similarity(a, pronounce[e][j]))
                prob3.append(1-(jellyfish.levenshtein_distance(a,pronounce[e][j]))/7)
                prob.append(prob3[j]+(prob1[j]+prob2[j])/2)
            probability.append(prob)
    return probability



def word_change(): #오류단어를 교체
    global correct_word
    correct_word=[]
    if probability==[]:
        return jamak
    else:
        change_word=[]
        for i in range(len(probability)):
            err_word_index=probability[i].index(max(probability[i]))
            correct_word.append(word_list[i][err_word_index])
            
        for a in range(len(jamak)):
            for b in range(len(error_word)):
                if jamak[a] == error_word[b]:
                    jamak[a]=correct_word[b]
    return jamak


In [7]:
f=open(r'C:\Users\IBK\Desktop\psychology_lecture.txt',encoding="utf8") # 자막이 들어올 위치. 자막을 한 줄씩 받을 예정

while True:
    line=f.readline()
    jamak_nn=tagger.nouns(line) #자막에 나오는 의미있는 명사를 찾기
    
    if not line:
        break
    
    jamak_nouns=[]
    for w in jamak_nn:
        if w not in except_words:
            jamak_nouns.append(w)
    
    if tagger.morphs(line)!=[]: #자막문장을 품사별로 끊기
        jamak=tagger.morphs(line)
        
    error_word=find_error_word()
    check_nouns=nearby_error_word()
    word_list=check_word_list()
    pronounce=romanizing()
    probability=similarity()
    change_word=word_change()
    jamak=''.join(jamak)
    print(spacing(jamak),'\n')
    
f.close()


우리 교재에 있는 것처럼 사물을 지각하는 것은 감각기관을 통해서 경험에 의해서 그 외 우리가 지각이 내지 상응적 태어날 때부터 갖고 태어나게 되는 건 아니다 이렇게 이야기하는 

사물의 지각에 대한 그런 연구들이 있었습니다고 민들이 있었던 거죠 

그 다음에 이제 심리학에 그 영향을 입은 생리학적인 연계생리학적 영향이다고 하는 것은 이제 사람에 대해서 

사람의 신체 그리고 이제 사람의 어떤 그 귀능 특별히 여기서 이야기할 때 뇌라고 하는 것을 대단히 중요하게 생각을 하구요 

그 다음에 이제 우리가 심리적인 영향을 주는 사람의 신체는 호르몬이라고 하는 것들이 중요하게 생각합니다 

그러니까 이 호르몬이 몸에서 움직이면 호르몬이구요똑같은 물질이 우리의 뇌로 들어가면 신경전달물질 

이렇게 뇌세포네 세포 사이를 연결해주는 신경전달물질이라고 하는 걸로 이렇게 분리됩니다 

똑같은 똑같은 이제 그쉽게 말하면 똑같은 기능을 하는 건데 몸에서 기능하면 호르몬 뇌에서 기능하면 신경전달을 

이렇게 똑같은 걸 갖고 이렇게 명칭도 다르게 부르는 그런 특성들이 있습니다 

왜 그렇게 부르냐면 뭐라고 부르면 이건 몸에서 기능하는 거구나 뭐라고 부르면 아이게 뇌에서 기능하는 거구나 음 

이렇게 구분하기 위해서 그렇게 부르는 연 

그러니까 이제 이런 것들이 이제 전체적으로 사람의 몸에 대한 과학적인 연구가 증가하면 증가할수록 

사람의 심리를 이해하는 일도 훨씬 더 영향을 많이 줍니다 해서 그 영향으로 인해서 우리가 이제 어떤 뭐 

심리적인 문제가 장흔한 건데 우울하다 이러면 병원에 가서 약소음 우울한 기분 기운이 딱 없어지는 

이런 한 것들이 이제 이런 그 생리학적인 연구를 통해서 나온 결과물이다라고 보시면 됩니다 

